In [3]:
import smartsheet
from docxtpl import DocxTemplate
from num2words import num2words
from datetime import datetime
import locale
from jinja2 import Template
import pandas as pd
import numpy as np


# Configura tu acceso a Smartsheet
ACCESS_TOKEN = 'Jg2fmXeRVSlecIiTohGY2UAJr4YKycUs7cEfX'
SHEET_ID = '2152478669754244'

ss_client = smartsheet.Smartsheet(ACCESS_TOKEN)
sheet = ss_client.Sheets.get_sheet(SHEET_ID)

# Definir los IDs de las columnas basados en los proporcionados
COLUMN_ID_FOR_PROJECT_NAME = 527835979796356
COLUMN_ID_FOR_DATE = 8461911885827972
COLUMN_ID_FOR_PRONOUN = 7121570343636868
COLUMN_ID_FOR_COMPANY = 7283235420852100
COLUMN_ID_FOR_PROVINCE = 1653735886638980
COLUMN_ID_FOR_CANTON = 6157335514009476
COLUMN_ID_FOR_PARISH = 3905535700324228
COLUMN_ID_FOR_ADDRESS = 8409135327694724
COLUMN_ID_FOR_METRAJE = 3342585746902916
COLUMN_ID_FOR_WATER_METER = 8972085281116036
#COLUMN_ID_FOR_WATER_PAYMENT_METHOD = 6720285467430788
COLUMN_ID_FOR_LIGHT_METER = 4521262211878788
#COLUMN_ID_FOR_LIGHT_PAYMENT_METHOD = 17662584508292
COLUMN_ID_FOR_LANDLORD_NAME = 6773062025564036
COLUMN_ID_FOR_LANDLORD_ID = 3955193751490436
COLUMN_ID_FOR_LANDLORD_COMPANY_NAME = 5647162118721412
COLUMN_ID_FOR_EMAIL = 3395362305036164
#COLUMN_ID_FOR_PHONE = 7898961932406660
COLUMN_ID_FOR_RENT_AMOUNT = 6210112072142724
COLUMN_ID_FOR_GUARANTEE_AMOUNT = 3958312258457476
COLUMN_ID_FOR_CONTRACT_END_DATE = 299137561218948
COLUMN_ID_FOR_WHO_INVOICE = 1425037468061572
COLUMN_ID_FOR_BENEFICIARY_BANK = 5420177937354628
COLUMN_ID_FOR_ACCOUNT_TYPE = 6229487214874500
COLUMN_ID_FOR_BANK_ACCOUNT = 4575894741143428
COLUMN_ID_FOR_SUBLEASE_PERMISSION = 2410199886548868
COLUMN_ID_FOR_JURISDICTION = 4661999700234116
COLUMN_ID_FOR_WATER_CONDITION = 2216685840060292
COLUMN_ID_FOR_LIGHT_CONDITION = 4468485653745540
COLUMN_ID_FOR_TYPE_PERSON = 2269462398193540
#REPRESENTANTE LEGAL
COLUMN_ID_FOR_REPRESENTATIVE = 4580843801759620 #Si o No
COLUMN_ID_FOR_REPRESENTATIVE_NAME = 2126845517713284
COLUMN_ID_FOR_REPRESENTATIVE_ID = 7984675781037956
COLUMN_ID_FOR_REPRESENTATIVE_TITLE = 1071365894655876
COLUMN_ID_FOR_REPRESENTATIVE_PRONOUN = 1844395583426436

# Cargar la plantilla de contrato
template = DocxTemplate("templated.docx")

# Funciones de ayuda para reemplazar variables

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

def get_date_parts(date_str):
    if date_str:
        date = datetime.strptime(date_str.split('T')[0], '%Y-%m-%d')
        return date.day, date.strftime('%B'), date.year
    return None, None, None

def format_currency(amount):
    if amount is None or amount == '':
        return "N/A"
    try:
        amount = float(amount)
        words = num2words(amount, to='currency', lang='es').replace('euros', 'dólares').replace('euro', 'dólar').replace('céntimos', 'centavos')
        return f"{words} ({amount:,.2f} USD)"
    except ValueError:
        return amount

def get_title(pronoun):
    titles = {
        'el': 'señor',
        'la': 'señora',
        'los': 'señores',
        'las': 'señoras'
    }
    return titles.get(pronoun, '')

def get_title2(pronoun):
    titles = {
        'el': 'Sr.',
        'la': 'Srta.',
        'los': 'Sres.',
        'las': 'Sras.'
    }
    return titles.get(pronoun, '')

def get_arr(pronoun):
    arr = {
        'el': 'EL ARRENDADOR',
        'la': 'LA ARRENDADORA',
        'los': 'LOS ARRENDADORES',
        'las': 'LAS ARRENDADORAS'
    }
    return arr.get(pronoun, '')

def get_deARR(pronoun):
    deARR = {
        'el': 'del ARRENDADOR',
        'la': 'de la ARRENDADORA',
        'los': 'de los ARRENDADORES',
        'las': 'de las ARRENDADORAS'
    }
    return deARR.get(pronoun, '')

def get_sinPronARR(pronoun):
    arr = {
        'el': 'ARRENDADOR',
        'la': 'ARRENDADORA',
        'los': 'ARRENDADORES',
        'las': 'ARRENDADORAS'
    }
    return arr.get(pronoun, '')

def get_quien(pronoun):
    return 'quien' if pronoun in ['el', 'la'] else 'quienes'

def get_oas(pronoun):
    return 'o' if pronoun in ['el', 'la'] else 'rios' if pronoun == 'los' else 'rias'

def get_ns(pronoun):
    return '' if pronoun in ['el', 'la'] else 'n'

def get_company_description(company):
    descriptions = {
        'ECONOFARM': 'ECONOFARM S.A. con RUC número 1791715772001',
        'FARCOMED': 'FARMACIAS Y COMISARIATOS DE MEDICINAS S.A. FARCOMED con RUC número 1790710319001',
        'OKIDOKI': 'FARMACIAS Y COMISARIATOS DE MEDICINAS S.A. FARCOMED con RUC número 1790710319001'
    }
    return descriptions.get(company, '')

def get_purpose(company):
    purposes = {
        'ECONOFARM': 'CONSTITUIR UNA FARMACIA, a través de la cual se comercializará productos farmacéuticos, medicinales y de aseo',
        'FARCOMED': 'CONSTITUIR UNA FARMACIA, a través de la cual se comercializará productos farmacéuticos, medicinales y de aseo',
        'OKIDOKI': 'LA IMPLEMENTACIÓN DE UN MINIMARKET, a través del cual se comercializará comida y bebida'
    }
    return purposes.get(company, '')

def get_purpose2(company):
    purposes = {
        'ECONOFARM': 'productos farmacéuticos',
        'FARCOMED': 'productos farmacéuticos',
        'OKIDOKI': 'comida y/o bebida'
    }
    return purposes.get(company, '')

def get_sublease(permission):
    return 'EL ARRENDATARIO podrá subarrendar parcial y/o totalmente el inmueble arrendado y destinarlo a objeto distinto al especificado en la cláusula anterior, según acuerdo expreso con EL ARRENDADOR' if permission == 'SI' else 'EL ARRENDATARIO no podrá subarrendar parcial y/o totalmente el inmueble arrendado'

def get_bank_prefix(bank_name):
    if not isinstance(bank_name, str):
        return ""
    bank_name_lower = bank_name.lower()
    if "banco" in bank_name_lower:
        return "del"
    elif "produbanco" in bank_name_lower:
        return "de"
    else:
        return "la"

def get_title_representative(pronoun):
    titles = {
        'el': 'señor',
        'la': 'señora',
        'los': 'señores',
        'las': 'señoras'
    }
    return titles.get(pronoun, '')

# Variables para datos
data = {}
project_name = ""
fecha_fin_contrato = None
fecha_inicio_contrato = None
representante = None
representante_nombre = None
representante_id = None
representante_cargo = None
tipo_persona = None
representante_pronoun = None
representante_titulo = None

for row in sheet.rows:
    for cell in row.cells:
        if cell.column_id == COLUMN_ID_FOR_CANTON:
            data['Ciudad'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_DATE:
            fecha_inicio_contrato = cell.value
            data['FechaInicioContrato'] = fecha_inicio_contrato
            dia, mes, anno = get_date_parts(cell.value)
            data['Dia'] = dia
            data['Mes'] = mes
            data['Anno'] = anno
        elif cell.column_id == COLUMN_ID_FOR_PRONOUN:
            pronoun = cell.value
            data['PronombreArrendador'] = pronoun
            data['TituloArrendador'] = get_title(pronoun)
            data['Quien'] = get_quien(pronoun)
            data['ARR'] = get_arr(pronoun)
            data['OAS'] = get_oas(pronoun)
            data['Ns'] = get_ns(pronoun)
            data['TituloArrendador2'] = get_title2(pronoun)
            data['deARR'] = get_deARR(pronoun)
            data['sinPronARR'] = get_sinPronARR(pronoun)
        elif cell.column_id == COLUMN_ID_FOR_PROJECT_NAME:
            project_name = cell.value
        elif cell.column_id == COLUMN_ID_FOR_COMPANY:
            data['CIA'] = get_company_description(cell.value)
            data['Proposito'] = get_purpose(cell.value)
            data['Purpose2'] = get_purpose2(cell.value)
        elif cell.column_id == COLUMN_ID_FOR_PROVINCE:
            data['Provincia'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_PARISH:
            data['Parroquia'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_ADDRESS:
            data['Direccion'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_METRAJE:
            data['m2'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_LANDLORD_NAME:
            data['ArrendadorNombre'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_LANDLORD_ID:
            data['ArrendadorCedula'] = str(cell.value).replace('.0', '')
        elif cell.column_id == COLUMN_ID_FOR_TYPE_PERSON:
            tipo_persona = cell.value
        elif cell.column_id == COLUMN_ID_FOR_LANDLORD_COMPANY_NAME:
            data['NombreEmpresa'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_EMAIL:
            data['Correo'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_RENT_AMOUNT:
            data['CANON'] = format_currency(cell.value)
        elif cell.column_id == COLUMN_ID_FOR_GUARANTEE_AMOUNT:
            data['Garantia'] = format_currency(cell.value)
        elif cell.column_id == COLUMN_ID_FOR_CONTRACT_END_DATE:
            fecha_fin_contrato = cell.value
            data['FechaFinContrato'] = fecha_fin_contrato
        elif cell.column_id == COLUMN_ID_FOR_BANK_ACCOUNT:
            data['CtaBancaria'] = str(cell.value).replace('.0', '')
        elif cell.column_id == COLUMN_ID_FOR_SUBLEASE_PERMISSION:
            data['Subarriendo'] = get_sublease(cell.value)
        elif cell.column_id == COLUMN_ID_FOR_LIGHT_CONDITION:
            medidor_luz = cell.value
            if medidor_luz == "Uso medidor que se encuentra en PDV de manera provisional hasta gestionar nuevo medidor":
                texto_luz_template = ("1.- Para el servicio básico de electricidad se gestionará un nuevo medidor por parte del ARRENDATARIO"
                                    "y lo cancelará mensualmente, así mismo una vez termine la vigencia del contrato "
                                    "deberá realizar el trámite respectivo para su anulación.")
            else:
                texto_luz_template = ("1.- El servicio básico de electricidad con el medidor No. {{NroLuz}} a nombre de "
                                    "{{ArrendadorNombre}}, EL ARRENDATARIO lo cancelará mensualmente previa presentación "
                                    "de la planilla.")
        elif cell.column_id == COLUMN_ID_FOR_LIGHT_METER:
            data['NroLuz'] = str(cell.value).replace('.0', '')
        elif cell.column_id == COLUMN_ID_FOR_WATER_CONDITION:
            medidor_agua = cell.value
            if medidor_agua == "Uso medidor que se encuentra en PDV de manera provisional hasta gestionar nuevo medidor":
                texto_agua_template = ("2.- Para el servicio básico de agua potable se gestionará un nuevo medidor por parte del ARRENDATARIO"
                                    "y lo cancelará mensualmente, así mismo una vez termine la vigencia del contrato "
                                    "deberá realizar el trámite respectivo para su anulación.")
            else:
                texto_agua_template = ("2.- El consumo de agua potable con el medidor No. {{NroAgua}}, EL ARRENDATARIO "
                                    "lo cancelará mensualmente previa presentación de la planilla.")
        elif cell.column_id == COLUMN_ID_FOR_WATER_METER:
            data['NroAgua'] = str(cell.value).replace('.0', '')
        elif cell.column_id == COLUMN_ID_FOR_WHO_INVOICE: 
            data['NombreFactura'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_BENEFICIARY_BANK:
            banco = cell.value
            data['dBanco'] = get_bank_prefix(banco)
            data['Banco'] = banco
        elif cell.column_id == COLUMN_ID_FOR_ACCOUNT_TYPE: 
            data['TipoCuenta'] = cell.value
        elif cell.column_id == COLUMN_ID_FOR_JURISDICTION: 
            data['Jurisdiccion'] = cell.value

        elif cell.column_id == COLUMN_ID_FOR_REPRESENTATIVE:
            representante = cell.value
        elif cell.column_id == COLUMN_ID_FOR_REPRESENTATIVE_PRONOUN:
            representante_pronoun = cell.value    
            representante_titulo = get_title_representative(cell.value)
        elif cell.column_id == COLUMN_ID_FOR_REPRESENTATIVE_NAME:
            representante_nombre = cell.value
        elif cell.column_id == COLUMN_ID_FOR_REPRESENTATIVE_ID:
            representante_id = str(cell.value).replace('.0', '')
        elif cell.column_id == COLUMN_ID_FOR_REPRESENTATIVE_TITLE:
            representante_cargo = cell.value
            if representante == "Si":
                if tipo_persona == "Jurídica":
                    texto_repte_tpl = "{{PronombreArrendador}} {{TituloArrendador}} {{ArrendadorNombre}} de nacionalidad ecuatoriana, mayor de edad, portador de la cédula de ciudadanía No. {{ArrendadorCedula}} en calidad de representante legal de la empresa {{NombreEmpresa}}"
                else:
                    texto_repte_tpl = "{{PronombreArrendador}} {{TituloArrendador}} {{ArrendadorNombre}} de nacionalidad ecuatoriana, mayor de edad, portador de la cédula de ciudadanía No. {{ArrendadorCedula}} en calidad de representante legal de la empresa {{NombreEmpresa}}"
            else:
                texto_repte_tpl = "{{PronombreArrendador}} {{TituloArrendador}} {{ArrendadorNombre}} de nacionalidad ecuatoriana, mayor de edad, portador de la cédula de ciudadanía No. {{ArrendadorCedula}}"
        elif fecha_inicio_contrato and fecha_fin_contrato:
            start_date = datetime.strptime(fecha_inicio_contrato, '%Y-%m-%d')
            end_date = datetime.strptime(fecha_fin_contrato, '%Y-%m-%d')
            diff_in_years = (end_date - start_date).days / 365.25
            data['Plazo'] = round(diff_in_years)

        data['TestTabla'] = np.random.randint(1, 101, size=(4, 2))


#Jinja2 para variables anidadas
template_luz = Template(texto_luz_template)
data['TextoLuz'] = template_luz.render(NroLuz=data.get('NroLuz', ''), ArrendadorNombre=data.get('ArrendadorNombre', ''))

template_agua = Template(texto_agua_template)
data['TextoAgua'] = template_agua.render(NroAgua=data.get('NroAgua', ''))

template_persona = Template(texto_repte_tpl)
data["TipoPersona"] = template_persona.render(PronombreArrendador=data.get("PronombreArrendador", ""), TituloArrendador=data.get("TituloArrendador", ""), 
                                              ArrendadorNombre=data.get("ArrendadorNombre",""), ArrendadorCedula=data.get("ArrendadorCedula", ""), NombreEmpresa=data.get("NombreEmpresa", ""))

# Reemplazar valores en el documento
template.render(data)
output_filename = f"Contrato Borrador {project_name}.docx"
template.save(output_filename)

print(f"Documento generado: {output_filename}")



Documento generado: Contrato Borrador TEST ALEX 22.docx
